In [1]:
import pandas as pd
import numpy as np

data =  pd.read_csv("train.csv")
data.dtypes

PassengerId      int64
Survived         int64
Pclass           int64
Name            object
Sex             object
Age            float64
SibSp            int64
Parch            int64
Ticket          object
Fare           float64
Cabin           object
Embarked        object
dtype: object

#### Feature selection
Name? Dropped for now

Ticket? Dropped for now

TODO: Need to have deeper insights to use these features. And also will have to do some text processing.

In [2]:
(x,y) = data.shape
data.isna().sum()/x *100

PassengerId     0.000000
Survived        0.000000
Pclass          0.000000
Name            0.000000
Sex             0.000000
Age            19.865320
SibSp           0.000000
Parch           0.000000
Ticket          0.000000
Fare            0.000000
Cabin          77.104377
Embarked        0.224467
dtype: float64

Cabin feature is missing values for 77% of dataset. Dropped.

Age:
    1. Remove rows
    2. Impute with mean
    3. Impute using advanced methods (K-nn)

In [3]:
data_t = data.drop(['Name','Ticket','Cabin'],axis=1)
data_t.head()

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,1,0,3,male,22.0,1,0,7.2500,S
1,2,1,1,female,38.0,1,0,71.2833,C
2,3,1,3,female,26.0,0,0,7.9250,S
3,4,1,1,female,35.0,1,0,53.1000,S
4,5,0,3,male,35.0,0,0,8.0500,S


In [4]:
data_1 = data_t.copy()
# Removing missing rows
#data_1 = data_1.dropna(subset = ['Age'])

#imputing median
# print(data_1.Age.median())
# data_1.Age = data_1.Age.fillna(data_1.Age.median())

#imputing mean
# print(data_1.Age.mean())
# data_1.Age = data_1.Age.fillna(data_1.Age.mean())

#imputing mode
print(data_1.Age.mode()[0])
data_1.Age = data_1.Age.fillna(data_1.Age.mode()[0])
data_1.Embarked = data_1.Embarked.fillna(data_1.Embarked.mode()[0])

24.0


In [5]:
data_1['Sex']=data_1['Sex'].astype('category')
data_1['Sex']=data_1['Sex'].cat.codes

#data_1['Embarked']=data_1['Embarked'].astype('category')
#data_1['Embarked']=data_1['Embarked'].cat.codes

data_1 = pd.get_dummies(data_1,columns=['Embarked'])

data_1.columns

Index(['PassengerId', 'Survived', 'Pclass', 'Sex', 'Age', 'SibSp', 'Parch',
       'Fare', 'Embarked_C', 'Embarked_Q', 'Embarked_S'],
      dtype='object')

In [6]:
# excluded PassengerId,Name,Ticket,Cabin
# X=data_1[['Pclass','Sex','Age','SibSp','Parch','Fare','Embarked']]
X=data_1[['Pclass','Sex','Age','SibSp','Parch','Fare','Embarked_C', 'Embarked_Q', 'Embarked_S']]
y=data_1['Survived']

# split X and y into training and testing sets
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.25,random_state=0)
# import the class

from sklearn.linear_model import LogisticRegression

# instantiate the model (using the default parameters)
logreg = LogisticRegression(solver='lbfgs')
# fit the model with data
logreg.fit(X_train,y_train)

y_pred=logreg.predict(X_test)

#calculate matrix
from sklearn import metrics
cnf_matrix = metrics.confusion_matrix(y_test, y_pred)
print(cnf_matrix)
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))
print("Precision:",metrics.precision_score(y_test, y_pred))
print("Recall:",metrics.recall_score(y_test, y_pred))

[[117  22]
 [ 23  61]]
Accuracy: 0.7982062780269058
Precision: 0.7349397590361446
Recall: 0.7261904761904762


/home/mldev/.local/lib/python3.5/site-packages/sklearn/linear_model/logistic.py:757: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)


### Age missing rows removed
[[85 18]
 [18 58]]
 
Accuracy: 0.7988826815642458
Precision: 0.7631578947368421
Recall: 0.7631578947368421

### Age Imputing median/mean

[[116  23]
 [ 23  61]]
 
Accuracy: 0.7937219730941704
Precision: 0.7261904761904762
Recall: 0.7261904761904762

### Age imputing mode
[[115  24]
 [ 20  64]]
 
Accuracy: 0.8026905829596412
Precision: 0.7272727272727273
Recall: 0.7619047619047619

## Resorting to mode for now


### Embarked one hot encoded

[[117  22]
 [ 23  61]]
 
Accuracy: 0.7982062780269058
Precision: 0.7349397590361446
Recall: 0.7261904761904762

In [33]:
test_data = pd.read_csv('test.csv')
test_data.head(20)

# Preparing test data for prediction
test_data_t = test_data.drop(['Name','Ticket','Cabin'],axis=1)
test_data_1 = test_data_t.copy()

test_data_1.isna().sum()

#imputing mode
test_data_1.Age = test_data_1.Age.fillna(data_1.Age.mode()[0])
test_data_1.Fare = test_data_1.Fare.fillna(data_1.Fare.mode()[0])
test_data_1.Embarked = test_data_1.Embarked.fillna('S')

test_data_1['Sex']=test_data_1['Sex'].astype('category')
test_data_1['Sex']=test_data_1['Sex'].cat.codes
test_data_1 = pd.get_dummies(test_data_1,columns=['Embarked'])

test_X=test_data_1[['Pclass','Sex','Age','SibSp','Parch','Fare','Embarked_C', 'Embarked_Q', 'Embarked_S']]
test_Y=test_data_1['PassengerId']

predictions = logreg.predict(test_X)

output = pd.DataFrame({ 'PassengerId' : test_Y, 'Survived': predictions })
output.to_csv('submission.csv', index=False)
